# How well does a conditional LSTM generate molecules which have desired characteristics?

In [20]:
import sys
sys.path.insert(0, '..')

import matplotlib.pyplot as plt

from rdkit import Chem
from pathlib import Path


import pickle, gzip, torch
import numpy as np

import warnings

from rnn_utils import load_rnn_model
import action_sampler, rnn_sampler
from gencond.properties import PROPERTIES, TYPES
%matplotlib inline

In [21]:

'''
model_path = './output/ZINC/batch_size_60_2/model_90_0.213.pt' 
#model_path = '../our_model/output/ZINC/batch_size_60_2/model_72_0.219.pt'
# zinc
test_smiles, test_prop = np.load("../../data/ZINC/ZINC_clean_smi_test_smile.npz", allow_pickle=True).values()
train_smiles, train_prop = np.load("../../data/ZINC/ZINC_clean_smi_train_smile.npz", allow_pickle=True).values()
train_smiles = train_smiles.tolist()


def normalize(prop):
    train_smiles, train_prop = np.load("../../data/ZINC/ZINC_clean_smi_train_smile.npz",allow_pickle=True).values()
    mean = np.mean(train_prop, axis = 0)
    std = np.std(train_prop, axis = 0)
    return (prop - mean) / std

#validation_smiles = train_smiles[:10000]
#def normalize_augmented(prop):
#    _, train_prop = np.load("../../data/QM9/smiles_data_augmentation.npz", allow_pickle=True).values()
#    mean = np.mean(train_prop, axis = 0)
#    std = np.std(train_prop,axis =0)
#    return (prop - mean)/ std

'''


#QM9
model_path = './output/QM9/2020_1_17/model_79_0.167.pt' 
test_smiles, test_prop = np.load("../../data/QM9/QM9_clean_smi_test_smile.npz", allow_pickle=True).values()
train_smiles, train_prop = np.load("../../data/QM9/QM9_clean_smi_train_smile.npz", allow_pickle=True).values()
val_smiles, val_prop = train_smiles[:10000],train_prop[:10000,:]
train_smiles, train_prop = train_smiles[10000:],train_prop[10000,:]
train_smiles = train_smiles.tolist()
print('training smiles num:', len(train_smiles))
print('validation smiles num:', val_smiles.shape)
print('test smiles num:', test_smiles.shape)


def normalize(prop):
    train_smiles, train_prop = np.load("../../data/QM9/QM9_clean_smi_train_smile.npz",allow_pickle=True).values()
    mean = np.mean(train_prop, axis = 0)
    std = np.std(train_prop, axis = 0)
    return (prop - mean) / std

'''
## load gucamol train smiles
#def normalize(prop):
    #mean, std = np.load('../../data/Guacamol/normalizer.npy',allow_pickle=True)
    #return (prop - mean) / std

def normalize(prop):    
    s, y = np.load("../../data/Guacamol/train_props.npz").values()
    mean = np.mean(y, axis = 0)
    std = np.std(y, axis = 0)
    return (prop - mean)/std

train_smiles = []
for row in open("../../data/Guacamol/guacamol_v1_train.smiles"):
    row = row.strip()
    train_smiles.append(row)   
    
#guacamol
test_smiles, test_prop = np.load("../../data/Guacamol/test_props.npz").values()
train_smile, train_prop = np.load("../../data/Guacamol/train_props.npz").values()

model_path = '../our_model_icml/output/Guacamol/continued/batch_size_100/model_12_0.140.pt' 
#model_path = './output/Guacamol/baobab/model_91_0.132.pt'
print('training smiles num:', len(train_smiles))
#print('validation smiles num:', val_smiles.shape)
print('test smiles num:', test_smiles.shape)   
'''
model_def = Path(model_path).with_suffix('.json')
model = load_rnn_model(model_def, model_path, device='cpu', copy_to_cpu=True)
sampler = rnn_sampler.ConditionalSmilesRnnSampler(device='cpu')


training smiles num: 113885
validation smiles num: (10000,)
test smiles num: (10000,)


# Test on random subset of 1000 molecules in test set

For each of the 1000 molecules, compute their properties, and then sample from the LSTM 10 SMILES strings conditioned on these property values.

In [22]:
len(train_smiles)

113885

In [23]:
#np.random.seed(42)
subset = np.random.permutation(len(test_smiles)) #[:1000]
#subset = subset[:10000]
#subset = np.random.permutation(1000)
sample_size = 10
test_smiles = test_smiles[subset]
test_properties = test_prop[subset]
generated = sampler.sample(model, normalize(test_properties), sample_size)
#generated = sampler.sample(model, prior.transform(test_properties), sample_size)

In [ ]:
test_smiles.shape

In [ ]:
##for figure2 
### to generate multiple samples for one target proop
np.random.seed(1)
#subset = np.random.permutation(len(test_smiles)) #[:1000]
#subset = np.random.permutation(10000)
k = 11
sample_size = 10
target_smiles = test_smiles[k:k+1]
target_properties = test_prop[k:k+1]
generated = sampler.sample(model, normalize(target_properties), sample_size)
#generated = sampler.sample(model, prior.transform(test_properties), sample_size)

prop = []
canonical = []
for i in range(len(generated[0])):
    sample = generated[0][i]
    mol = Chem.MolFromSmiles(sample)
    try:
        prop.append([p(mol) for p in PROPERTIES.values()])
        canonical.append(Chem.MolToSmiles(mol))
    except:
            pass # m    

for i in range(len(canonical)):
    print(i)
    plt.scatter(np.arange(9), np.array(prop[i]).round(3).flatten()) #alpha=1, s =80)
    
plt.scatter(np.arange(9), target_properties.round(3).flatten(),color='red',alpha=1)
plt.xlabel('property dimension index')
plt.ylabel('property value')  

from rdkit import Chem
from rdkit.Chem import Draw

sampled_mols = [Chem.MolFromSmiles(canonical[i]) for i in set(np.arange(len(canonical)))]
Draw.MolsToGridImage(sampled_mols, molsPerRow=5, subImgSize=(300, 300))


m = Chem.MolFromSmiles(target_smiles[0])
Draw.MolToImage(m)
len(np.unique(np.array(canonical)))
train_mol = list(set(train_smiles).intersection(set(canonical))) # remove the validation set
sampled_mols = [Chem.MolFromSmiles(train_mol[i]) for i in set(np.arange(len(train_mol)))]
Draw.MolsToGridImage(sampled_mols, molsPerRow=5, subImgSize=(300, 300))n

# Check validity, novelty and unicity

In [24]:
from itertools import chain
valid_mol = []
num_valid = 0
for row in range(generated.shape[0]):
    valid = []
    for samples in generated[row]:
        mol = Chem.MolFromSmiles(samples)
        try:
            valid.append(Chem.MolToSmiles(mol))
            num_valid = num_valid + 1
            
        except:
            pass # molecule invalid
    valid_mol.append(valid)    
validity_ = num_valid / (len(generated) * generated[0].shape[0])
print('validity')
print('unicity')
print('noelty')                                  

print(np.around(validity_, decimals=6))
str_list = [x for x in valid_mol if x != []]
valid_mol_list = list(chain.from_iterable(str_list))
num_unique = len(np.unique(np.array(valid_mol_list), return_index=True)[1])
print(np.around(num_unique / num_valid, decimals=6))

#common = list(set(train_smiles.intersection(valid_mol_list)))
common = list(set(train_smiles).intersection((valid_mol_list)))  # remove the validation set
novelty_= (num_valid - len(common)) / num_valid 
print(np.around(novelty_, decimals = 6))

validity
unicity
noelty
0.9642
0.558276
0.645831


## Compute the properties of these generated smiles strings

In [25]:
simulated = []
smiles_matches = []
index = []
for row in range(generated.shape[0]):
    prop = []
    canonical = []
    for sample in generated[row]:
        mol = Chem.MolFromSmiles(sample)
        try:
            prop.append([p(mol) for p in PROPERTIES.values()])
            canonical.append(Chem.MolToSmiles(mol))
        except:
            pass # molecule invalid
        
    if prop !=[]:
        index.append(row)
        
    simulated.append(np.array(prop))
    smiles_matches.append(np.sum([test_smiles == s for s in canonical]))
    

## First, check molecule validity, and confirm that we aren't just always recreating the same SMILES string we started with…

In [26]:
if simulated[0].shape[0]>1:
    mean_prop= [item.mean(axis = 0 ) for item in simulated]

else:
    mean_prop = simulated
    
valid_smiles_prop = np.array(mean_prop)[index]
valid_smiles_prop = np.vstack(valid_smiles_prop)
test_prop = test_properties[index]
corr_coef = []
for i in range(9):
    corr_coef.append(np.corrcoef(valid_smiles_prop[:,i], test_prop[:,i])[0,1])
    
print(np.around(corr_coef, decimals=6))

[0.996233 0.998001 0.994686 0.971792 0.996827 0.995805 0.993406 0.972188
 0.995973]


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: RuntimeWarning: Mean of empty slice.
  


In [27]:
MAE = np.mean(np.absolute(valid_smiles_prop - test_prop), axis = 0) 
print('Mean absolute error:', MAE)

Mean absolute error: [3.31287891e-02 1.87653686e-03 6.16217054e-02 9.80934375e-03
 8.81407379e-01 2.76334188e+00 3.18890006e-01 1.89471328e-03
 2.82933055e-02]


In [28]:
MSE_per_prop = np.mean(np.power(valid_smiles_prop - test_prop, 2),axis = 0) 
MSE_per_prop

array([9.18893099e-03, 6.53551246e-04, 1.09930722e-02, 3.22673092e-04,
       2.82087648e+00, 1.95609971e+01, 7.88064811e-01, 2.47439008e-03,
       1.18536828e-02])

In [29]:
MSE = np.mean(np.power(valid_smiles_prop - test_prop, 2)) 
print('MSE')
print(np.around(MSE, decimals = 6))

MSE
2.578381


In [30]:
MAE = np.mean(np.absolute(valid_smiles_prop - test_prop)) 
print('MAE')
print(np.around(MAE, decimals = 6))

MAE
0.455585


In [31]:
len(simulated)

10000

## Plot correlation between input to conditional LSTM model, and properties of generated molecules

In [ ]:
warnings.filterwarnings('ignore')
titles = list(PROPERTIES.keys())

plt.figure(figsize=(10,10))
for row in range(1000):
    prop = simulated[row]
    for i in range(prop.shape[-1]):
        plt.subplot(3,3,i+1)
        jitter = np.random.randn(prop.shape[0], 2) * (0.0 if TYPES[titles[i]] == float else 0.05)
        plt.plot(test_properties[row,i]+jitter[:,0], 
                 prop[:,i] + jitter[:,1], 
                 marker='.', linewidth=0, color='#3333cc', alpha=0.5)

for i in range(9):
    plt.subplot(3,3,i+1)
    a,b = plt.xlim()
    c,d = plt.ylim()
    lims = min(a,c), max(b,d)
    plt.xlim(lims)
    plt.ylim(lims)
    plt.title(titles[i] + '\n Correlation coef:\n ' +  str(corr_coef[i]))
    if i >= 6:
        plt.xlabel("input value")
    if i % 3 == 0:
        plt.ylabel("output value")
    
plt.tight_layout();

In [ ]:
test_properties.shape

# Test set log likelihood

In [ ]:
from torch.utils.data import DataLoader
from rnn_utils import load_smiles_from_list, get_tensor_dataset
from smiles_char_dict import SmilesCharDictionary
from atalaya import Logger
graph_logger = ''

#_, train_prop = np.load("../../data/QM9/QM9_clean_smi_train_smile.npz", allow_pickle=True).values()
mean = np.mean(train_prop, axis = 0)
std = np.std(train_prop,axis =0)


sd = SmilesCharDictionary()
criterion = torch.nn.CrossEntropyLoss(ignore_index=sd.pad_idx)

device = 'cpu'
test_s,_ = load_smiles_from_list(test_smiles, test_properties)
test_set = get_tensor_dataset(test_s, normalize(test_properties))
data_loader = DataLoader(test_set,batch_size=500,shuffle=False,num_workers=1,pin_memory=True)
from rnn_trainer import SmilesRnnTrainer

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), 0.01)
criterion = torch.nn.CrossEntropyLoss(ignore_index=sd.pad_idx)
trainer = SmilesRnnTrainer(graph_logger, model=model,
                                   normalizer_mean = mean,
                                   normalizer_std = std,
                                   criteria=[criterion],
                                   optimizer=optimizer,
                                   device=device,
                                   log_dir=None)

_logp = trainer.validate(data_loader, 1)

In [ ]:
print(np.round(_logp, decimals = 6))

In [ ]:
len(test_smiles)

# conditional generation performance over multiple runs on the full test set
### use L2 and L1 norm on the (y-y')

In [ ]:
## mean mse
test_smiles, test_prop = np.load("../../data/Guacamol/test_props.npz").values()
subset = np.random.permutation(len(test_smiles))
subset = subset[:10000]
test_smiles = test_smiles[subset]
test_properties = test_prop[subset,:]
MSE = []
MES_per_dim = []
MAE = []
MAE_per_dim = []
for i in range(10):
    #test_smiles, test_prop = np.load("../../data/QM9/QM9_clean_smi_test_smile.npz", allow_pickle=True).values()
    #test_smiles, test_prop = np.load("../../data/Guacamol/test_props.npz").values()
    np.random.seed(1)
    #subset = np.random.permutation(len(test_smiles))
    sample_size = 1
    
    generated = sampler.sample(model, normalize(test_properties), sample_size)

    simulated = []
    index = []
    for row in range(generated.shape[0]):
        prop = []
        canonical = []
        for sample in generated[row]:
            mol = Chem.MolFromSmiles(sample)
            try:
                prop.append([p(mol) for p in PROPERTIES.values()])
                canonical.append(Chem.MolToSmiles(mol))
            except:
                pass # molecule invalid

        if prop !=[]:
            index.append(row)

        simulated.append(np.array(prop))

    if simulated[0].shape[0]>1:
        mean_prop= [item.mean(axis = 0 ) for item in simulated]

    else:
        mean_prop = simulated

    valid_smiles_prop = np.array(mean_prop)[index]
    valid_smiles_prop = np.vstack(valid_smiles_prop)
    test_prop = test_properties[index]   

    mse = np.mean(np.power(valid_smiles_prop - test_prop, 2)) 
    mae = np.mean(np.absolute(valid_smiles_prop - test_prop)) 
    print('Mean square error:', mse)
    per_prop = np.mean(np.power(valid_smiles_prop - test_prop, 2),axis = 0)
    mae_per_prop = np.mean(np.absolute(valid_smiles_prop - test_prop),axis = 0)
    
    MSE.append(mse)
    MES_per_dim.append(per_prop)
    MAE.append(mae)
    MAE_per_dim.append(mae_per_prop)

In [ ]:
print('Mean square error')
print("%.4f" u"\u00B1" "%.4f"%(np.array(MSE).mean(), np.array(MSE).std()))
print('Mean absolute error')
print("%.4f" u"\u00B1" "%.4f"%(np.array(MAE).mean(), np.array(MAE).std()))

In [ ]:
print('Mean square error')
mean_per = np.array(MES_per_dim).mean(axis = 0)
std_per = np.array(MES_per_dim).std(axis = 0)
for i in range(9):
    print("%.4f" u"\u00B1" "%.4f" %(mean_per[i],std_per[i]))


In [ ]:
print('Mean absolute error')
MAE_mean_per = np.array(MAE_per_dim).mean(axis = 0)
MAE_std_per = np.array(MAE_per_dim).std(axis = 0)
for i in range(9):
    print("%.4f" u"\u00B1" "%.4f" %(MAE_mean_per[i],MAE_std_per[i]))


# statistics over multiple property

In [ ]:
#lstm
val_n = np.array([0.9619,0.9642,0.9701])
print("%.4f" u"\u00B1" "%.4f" %(val_n.mean(),val_n.std()))

unicity_n = np.array([0.9667,0.9694,0.96618])
print("%.4f" u"\u00B1" "%.4f" %(unicity_n.mean(),unicity_n.std()))

Nov_n = np.array([0.3660,0.3581,0.37315])
print("%.4f" u"\u00B1" "%.4f" %(Nov_n.mean(),Nov_n.std()))

_logp_n = np.array([0.21800,0.21447,0.21329])
print("%.4f" u"\u00B1" "%.4f" %(_logp_n.mean(),_logp_n.std()))

mse_n = np.array([10.337,9.7910,8.9395])
print("%.4f" u"\u00B1" "%.4f" %(mse_n.mean(),mse_n.std()))

In [ ]:
## our
val_n = np.array([0.9886,0.9863,0.9909])
print("%.4f" u"\u00B1" "%.4f" %(val_n.mean(),val_n.std()))

unicity_n = np.array([0.9629,0.9634,0.9641])
print("%.4f" u"\u00B1" "%.4f" %(unicity_n.mean(),unicity_n.std()))

Nov_n = np.array([0.2605,0.2676,0.2345])
print("%.4f" u"\u00B1" "%.4f" %(Nov_n.mean(),Nov_n.std()))

_logp_n = np.array([0.2357,0.2335,0.2437])
print("%.4f" u"\u00B1" "%.4f" %(_logp_n.mean(),_logp_n.std()))

mse_n = np.array([7.0603,8.4115,6.4833])
print("%.4f" u"\u00B1" "%.4f" %(mse_n.mean(),mse_n.std()))

# lets see if the hidden state represents well the mol structure

In [ ]:
from rnn_utils import rnn_start_token_vector
from torch.distributions import Categorical, Distribution
import torch.nn.functional as F
batch_size = 100
max_seq_length = 101
device = 'cpu'
hidden = model.init_hidden(batch_size, device)
inp = rnn_start_token_vector(batch_size, device)

actions = torch.zeros((batch_size, max_seq_length), dtype=torch.long).to(device)
distribution_cls= None
distribution_cls = Categorical if distribution_cls is None else distribution_cls

In [ ]:
properties = torch.FloatTensor(test_properties)
for char in range(max_seq_length):
    output, hidden = model(inp, properties, hidden)

    prob = F.softmax(output, dim=2)
    distribution = distribution_cls(probs=prob)
    action = distribution.sample()

    actions[:, char] = action.squeeze()

    inp = action

In [ ]:
inp = rnn_start_token_vector(batch_size, device)


In [ ]:
max_len = 100
device = 'cpu'
from rnn_utils import get_tensor_dataset, load_smiles_and_properties, set_random_seed
training_set="../../data/QM9/QM9_clean_smi_train_smile.npz"
train_seqs, train_prop = load_smiles_and_properties(training_set, max_len)
sample_indexs = np.arange(train_seqs.shape[0])

train_x, train_y = train_seqs[10000:,:], train_prop[10000:,:]
valid_x, valid_y = train_seqs[:10000,:], train_prop[:10000,:]

all_y = np.concatenate((train_y, valid_y), axis=0)
mean = np.mean(all_y, axis = 0)
std = np.std(all_y, axis = 0)

#np.save(args.data_path + '/normalizer.py', [mean, std])

train_y = (train_y - mean ) / std
valid_y = (valid_y - mean) / std

In [ ]:
batch_size = 1000
inp = torch.from_numpy(train_x[:batch_size,:-1]).long()
properties  = train_y[:batch_size,:]
properties = torch.FloatTensor(properties)

In [ ]:
hidden = model.init_hidden(inp.size(0), device)
output, hidden = model(inp, properties, hidden) 

In [ ]:
hidden[1][2].shape

In [ ]:
hidden[1][2].shape

In [ ]:
smiles_list, properties = np.load(training_set, allow_pickle=True).values()

In [ ]:
smiles_list = smiles_list[:batch_size]

In [ ]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem
from rdkit import DataStructs
tanimoto_sim = []
for i in range(batch_size):
    fp_A = AllChem.GetMorganFingerprint(Chem.MolFromSmiles(smiles_list[0]), 2)
    fp_B =  AllChem.GetMorganFingerprint(Chem.MolFromSmiles(smiles_list[i]), 2)
    sim = DataStructs.TanimotoSimilarity(fp_A,fp_B)
    tanimoto_sim.append(sim)

In [ ]:
np.array(tanimoto_sim).argsort()[-20:]

In [ ]:
fpA

In [ ]:
Draw.MolToImage(Chem.MolFromSmiles(smiles_list[0]))

In [ ]:
dist = []
target = hidden[1][2][0,:]
for i in range(batch_size):
    l_2_dist = torch.pow((hidden[1][2])[i,:] -  target, 2).mean()
    dist.append(l_2_dist)
   

In [ ]:
np.array(dist).argsort()[:20]

In [ ]:
properties = torch.FloatTensor(properties)
for char in range(max_seq_length):
    output, hidden = model(inp, properties, hidden)

    prob = F.softmax(output, dim=2)
    distribution = distribution_cls(probs=prob)
    action = distribution.sample()

    actions[:, char] = action.squeeze()
    inp = action